<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <img src="../images/QWorld.png"> </td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/pjr1363" target="_blank"> Paul Joseph Robin </a></td>
    </tr> 
 </table>
 
<hr>

## Higher Order Problems

Consider the following objective functions, where the variables are binary:

> $x_1 + x_2 - x_3$
>
> $x_1 x_2 + x_2 x_3 - x_3 x_1 - 3$
>
> $x_1 x_2 x_3 + x_2 x_3 x_1 - x_3 x_1 x_2$

These obejctive functions are $\textit{linear}$, $\textit{quadratic}$ and $\textit{cubic}$ respectively.

When the objective function contains terms of degree higher than $2$, it is called a Higher Order Binary Optimization (HOBO) problem. Depending on the dergree $k$ of this function, they are also known as $k-local$ problems. QUBO is a $2-local$ problem.

Due to the underlying architecture of the devices we work on, it is convenient to 'quadratize' higher order problems into binary problems (also known as $\textit{degree reduction}$). This is done by introducing additional variables ($\textit{ancillary/auxiliary variables}$), which in turn also increases the complexity.

---
## Quadratization

<!-- Intro --> 
There are many techniques for reducing the degree of the objective function, while keeping the number of variables under check. We will introduce the method proposed by Rosenberg in 1975, which involves repeated substitution of variables with an additional penalty term.

<!-- Algo --> 
### The Algorithm

1. Identify a product of two binary variables $x_i x_j$ which appears in a term with degree atleast 3.

2. Replace every instance of this product with another binary variable $y_{ij}$.

3. For a minimization problem, add a penalty term $C(x_i x_j − 2x_i y_{ij} − 2x_j y_{ij} + 3y_{ij} )$. For maximization problem, simply negate this term.

This penalty term enforces the quadratization constraint $x_i x_j = y_{ij}$, by going to zero. The constant $C$ is appropriately chosen to ensure a large value if this condition is not met.

---
### Task 1
Consider the following higher order problem and reformulate as QUBO problem:

$$f (x_1, x_2, x_3) = 5x_1 + 7x_1 x_2 - 3x_1 x_2 x_3$$


<a href="Higher_Order_Model_&_Quadratization_Solutions.ipynb#Task1">Click for the solution >></a>

---
## Optimising the penalty value $C$
**Aim**: Find a sufficiently large and minimal value of *C*.

**Why**: In a quantum annealer, we need to ensure that the penalty value *C* is 'just large enough' to ensure that the minimum value of energy configuration occurs only when the constraints are satisfied.

**How**: We use a 'hit and trial' method, evaluating different variables till the constraints are satisfied. Then, we prompt further values to check whether they violate the constraint(s). This is continued till we are confident that constraints are satisfied.

---
### Task 2
Conside the objective function of the previous task with a small change:
$$f (x_1, x_2, x_3) = 5x_2 + 7x_1 x_2 - 3x_1 x_2 x_3$$

Find the QUBO formulation and demonstrate that the minimum value of $C$ should be $4$.


<a href="Higher_Order_Model_&_Quadratization_Solutions.ipynb#Task2">Click for the solution >></a>


Different methods of quadratization and optimizing penalty values have been detailed in <a href="#Dattani">[Dattani]</a>


---
## Quadratic Reformulation of Max 3-SAT Problem

### Boolean Formula
A boolean formula is expressed as a conjugation of clauses, where each clause is a disjunction of literals. This representation is known as the Conjunctive Normal Form (CNF).

 - A conjunction in boolean algebra, $A \wedge B$ is true $iff$ both $A$ and $B$ are true, logically equivalent to $AND$ operator.
 - A disjunction $A \vee B$ is true unless both $A$ and $B$ are false, logically equivalent to $OR$ operator.
 - Literals are either boolean variables $x_i$ or their negated values $\overline{x_i}$, which are binary in nature.
 - Clauses are disjunctions of both positive $x_i$ and negated literals $\overline{x_i}$. 

### Boolean Satisfiability Problems

Also known as SAT problems, it determines the existence of an assignment of binary variables that satisfy a boolean formula. It was the first NP-Complete problem to be discovered. The significance of SAT is that, if there exists a deterministic polynomial-time algorithm for solving SAT, every NP-hard problem can be solved in polynomial time.
##### Representation
$$\psi = C_1 \wedge C_2 \wedge \cdots \wedge C_n$$
where $C_i = y_1 \vee y_2 \vee \cdots \vee y_k$ such that $y_k \in \{x_k, \overline{x_k}\}$.


#### 3-SAT Problem
3-SAT is a form of SAT problems where each clause has atmost three literals. Every SAT problem can be reduced to 3-SAT in the Conjunctive Normal Form. 
- eg: $(x_1 \vee x_2 \vee x_3) \wedge (\overline{x_1} \vee \overline{x_2} \vee x_3) \wedge (\overline{x_1} \vee x_2 \vee x_3)$ is a 3-SAT problem in CNF. 
The assignment $(x_1, x_2, x_3) = (0, 1, 1)$ is a solution of this 3-SAT.

---
### The Max 3-SAT

Max 3-SAT is an optimized version of the 3-SAT problem, with the goal of maximizing the number of satisfied clauses.

#### The Objective Function $g(X)$
The binary optimization methods we use, cannot accomodate CNF expressions. Hence, we design objective functions composed of binary variables. The method discussed below is based on penalties to achieve optimized values. See [[Kofler]](https://static.uni-graz.at/fileadmin/sowi/Working_Paper/2014-04_Kofler_Greistorfer_Wang_Kochenberger.pdf) for an extensive derivation.

##### The Four cases
Each clause of a Max 3-SAT problem contains atmost three clauses, each of which yields a different objective function.

- Zero Negations $(x_i \vee x_j \vee x_k)$
$$g(X) = x_i + x_j + x_k - x_i x_j - x_i x_k - x_j x_k + x_i x_j x_k $$

- One Negation $(x_i \vee x_j \vee \overline{x_k})$
$$g(X) = 1 - x_k + x_i x_k + x_j x_k - x_i x_j x_k $$

- Two Negations $(x_i \vee \overline{x_j} \vee \overline{x_k})$
$$g(X) = 1 - x_j x_k + x_i x_j x_k $$

- Three Negations $(\overline{x_i} \vee \overline{x_j} \vee \overline{x_k})$
$$g(X) = 1 - x_i x_j x_k $$

**such that $g(X) = 1$ if the corresponding clause is satisfied**.

Thus, a Max 3-SAT formulation with $n$ clauses can be transformed into the following Cubic Unconstrained Binary Optimization (CUBO) problem:
$$\sum_1^n g(X)_i$$ where $g(X)_i$ have elements from the set $\{x_i, x_j, x_k, 1\}$.


#### CUBO $\rightarrow$ QUBO

Rosenberg's method of quadratization is used to reduce this higher order model to QUBO.
- Introduce a set $Y$ of ancillary literals to replace higher degree terms such that
$$\text{max}\quad\sum_1^n g(X, Y)_i - C \sum_{ij\text{ in } Y}(x_i x_j − 2x_i y_{ij} − 2x_j y_{ij} + 3y_{ij} )$$ 
- Since each clause generates exactly one cubic term (3-SAT problem), this means that the augmented QUBO has at most $n$ ancillary literals.
- A Max 3-SAT formulation with $k$ literals and $n$ clauses yields QUBO formulations with atmost $n+k$ ancillary literals.
- The optimal solution to this QUBO formulation corresponds to the number of satisfied clauses $(\leq n)$ in Max 3-SAT formulation.

---
### Task 3
Given the following Max 3-SAT Formulation for $4$ literals and $2$ clauses:
$$(x_1 \vee x_2 \vee x_3) \wedge (x_1 \vee x_4 \vee \overline{x_3})$$

Convert this into a higher model formulation and quadratize. 

<a href="Higher_Order_Model_&_Quadratization_Solutions.ipynb#Task3">Click for the solution >></a>

---
## References
1. Efficient QUBO transformation for Higher Degree Pseudo Boolean Functions *(Amit Verma, Mark Lewis, Gary Kochenberger)* [[arXiv Preprint]](https://arxiv.org/abs/2107.11695v1)

2. Unconstrained Binary Models of the Travelling Salesman Problem Variants for Quantum Optimization *(Özlem Salehi, Adam Glos, Jaroslaw Adam Miszczak)* [[arXiv Preprint]](https://arxiv.org/abs/2106.09056)
<a id="Dattani"></a>
3. Quadratization in Discrete Optimization and Quantum Mechanics *(Nike Dattani)* [[arXiv Preprint]](https://arxiv.org/abs/1901.04405)
